# Assignment 3 - Tumor segmentation with uncertainties

### Course: Convolutional Neural Networks with Applications in Medical Image Analysis

For radiotherapy treatment, often multiple contrasts are acquired of the same anatomy to have more information for organ delineations. These contrasts have been explored in the previous assignment, and now, your task is to automate the organ delineation! The model to design and train might take any number of image contrasts as an input, and outputs a tumor segmentation. For this task, you will implement and use the [Dice score](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) to evaluate (aim for a Dice score higher than $0.8$ on the validation set).

For machine learning solutions to be reliable in real life scenarios, we may want to improve their explainability. Ideally, models should not only provide their predictions, but provide a motivation for them as well. One form of explainability is through uncertainty metrics, that evaluate how certain layers, and certain nodes affect the output predictions. A popular approach is called *Monte Carlo Dropout*, where during inference, output predictions are generated multiple times using differently initialized dropout layers, therefore the individual predictions will depend on different subsets of the layers' activations. The standard deviation in the generated outputs provides insight into the model's uncertainty about the individual pixels of the predictions, in our case, the segmentations.

Your task is to look through the highly customizable code below, which contains all the main steps for automated image segmentation from using only one contrast as an input. By changing the arrays of the DataGenerator, multiple contrasts can be added as input, similar as in the previous assignmnets. The most important issues with the current code are noted in the comments for easier comprehension. Your tasks, to include in the report, are:
- How you reached the required performances (a Dice score above $0.8$)
- Plot the training/validating losses and accuracies. Describe when to stop training, and why that is a good choice.
- Once you have reached the required loss on the validation data, only then evaluate your model on the testing data as well.
- Describe the thought process behind building your model and choosing the model hyper-parameters.
- Describe what you think are the biggest issues with the current setup, and how to solve them.
- Using the already implemented Monte Carlo Dropout model, explore the uncertainty of the predictions on the testing data.
- How does your estimated uncertainty depend on the dropout-rates used in the model?
- Look at the uncertainty of each image in the testing dataset. Do you see anything unusual in their values? If so explore the images!

Upload the updated notebook to Canvas by June $2^{nd}$ at 17:00.

Good luck and have fun!

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import random
random.seed(2023)

import numpy as np
np.random.seed(2023)  # Set seed for reproducibility

import tensorflow as tf
tf.random.set_seed(2023)

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras

from scipy.ndimage import zoom
import matplotlib.pyplot as plt

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print(f"GPU(s) available (using '{gpus[0].name}'). Training will be lightning fast!")
else:
    print("No GPU(s) available. Training will be suuuuper slow!")

# NOTE: These are the packages you will need for the assignment.
# NOTE: You are encouraged to use the course virtual environment, which already has GPU support.

##### The cell below will define the data generator for the data you will be using. You should not change anything in the below code!

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,
                 data_path,
                 arrays,
                 batch_size=32,
                 ):

        self.data_path = data_path
        self.arrays = arrays
        self.batch_size = batch_size

        if data_path is None:
            raise ValueError('The data path is not defined.')

        if not os.path.isdir(data_path):
            raise ValueError('The data path is incorrectly defined.')

        self.file_idx = 0
        self.file_list = [self.data_path + '/' + s for s in
                          os.listdir(self.data_path)]
        
        self.on_epoch_end()
        with np.load(self.file_list[0]) as npzfile:
            self.in_dims = []
            self.n_channels = 1
            for i in range(len(self.arrays)):
                im = npzfile[self.arrays[i]]
                im = zoom(im, 0.25)
                self.in_dims.append((self.batch_size,
                                    *np.shape(im),
                                    self.n_channels))

    def __len__(self):
        """Get the number of batches per epoch."""
        return int(np.floor((len(self.file_list)) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data."""
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) *
                               self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.file_list[k] for k in indexes]

        # Generate data
        a = self.__data_generation(list_IDs_temp)
        return a

    def on_epoch_end(self):
        """Update indexes after each epoch."""
        self.indexes = np.arange(len(self.file_list))
        np.random.shuffle(self.indexes)
    
    #@threadsafe_generator
    def __data_generation(self, temp_list):
        """Generate data containing batch_size samples."""
        # X : (n_samples, *dim, n_channels)
        # Initialization
        arrays = []

        for i in range(len(self.arrays)):
            arrays.append(np.empty(self.in_dims[i]).astype(np.single))

        for i, ID in enumerate(temp_list):
            with np.load(ID) as npzfile:
                for idx in range(len(self.arrays)):
                    x = npzfile[self.arrays[idx]] \
                        .astype(np.single)
                    if (np.max(x) > 0):
                        x /= np.max(x)
                    x = zoom(x, 0.25)
                    if (self.arrays[idx] == "mask"):
                        x = x > 0.5
                    x = np.expand_dims(x, axis=2)
                    arrays[idx][i, ] = x

        return arrays

# NOTE: Don't change the data generator!
# NOTE: There is now a resizing part of the images, this is to make training easier and faster.

In [ ]:
gen_dir = "/import/software/3ra023/vt23/brats/data/" # Change if you have copied the data locally on your machine 
array_labels = ['t2', 'mask']  # Available arrays are: 't1', 't1ce', 't2', 'flair', 'mask'.
batch_size = 16

gen_train = DataGenerator(data_path=gen_dir + 'training',
                          arrays=array_labels,
                          batch_size=batch_size)

gen_val = DataGenerator(data_path=gen_dir + 'validating',
                        arrays=array_labels,
                        batch_size=batch_size)

gen_test = DataGenerator(data_path=gen_dir + 'testing',
                         arrays=array_labels,
                         batch_size=batch_size)

# NOTE: What arrays are you using? You can use multiple contrasts as inputs, if you'd like. (STRONGLY encouraged)
# NOTE: What batch size are you using? Should you use more? Or less? What are the pros and cons?
# NOTE: Are you using the correct generators for the correct task? Training for training and validating for validating?

### Let's plot some example images from the dataset:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=(16.0, 8.0))
fig.subplots_adjust(left=0.001,
                    right=0.9975,
                    top=0.95,
                    bottom=0.005,
                    wspace=0.05,
                    hspace=0.14)

M, N = len(gen_train.arrays), 4
ax = []
for i in range(M):
    ax.append([None] * N)
    for j in range(N):
        ax[i][j] = plt.subplot2grid((M, N), (i, j), rowspan=1, colspan=1)

imgs = gen_train[0]
idx = np.random.randint(0, np.shape(gen_train[0][0])[0], 5)
ii = 0
for j in range(N):
    for i in range(M):
        im = ax[i][j].imshow(imgs[i][idx[ii], :, :, 0], cmap='gray', vmin=0, vmax=1)

        if j == 0:  # Label only on the left
            ax[i][j].set_ylabel(gen_train.arrays[i], fontsize=22)
        if j == N - 1:  # Colorbar only on the right
            divider = make_axes_locatable(ax[i][j])
            cax1 = divider.append_axes("right", size="7%", pad=0.05)
            cbar = plt.colorbar(im, cax=cax1)
    ii += 1    

### There's a chance, that the bottom row only shows black images, in that case, all four examples are from slices that don't have masks.

A quick summary of the data:

In [ ]:
# A quick summary of the data:

print(f"Number of training images : {len(gen_train.file_list)}")
print(f"Training batch size       : {gen_train.in_dims}")
found_masks = 0
for idx, (t2, mask) in enumerate(gen_val): # Add the additional inputs inside the brackets
    found_masks += np.sum(np.sum(mask > 0, (1, 2, 3)) > 0)
print(f"The percentage of slices that contain masks : {np.round(100 * (found_masks / (len(gen_val) * batch_size)), 2)}%")

### The dataset preprocessing so far has been to help you, you should not change anything above. However, from now on, take nothing for granted.

In [ ]:
# Import packages important for building and training your model.

import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import Flatten, Input
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from IPython.display import clear_output
%matplotlib inline

In [ ]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
                                    Add, AveragePooling2D, Flatten, Dense, UpSampling2D, Conv2DTranspose, Concatenate, LeakyReLU
from tensorflow.keras.models import Model

def build_generator(in_shape=(64, 64, 1)):
    num_filt = 4
    dropout_rate = 0.2
    inputs = Input(in_shape)
    # inputs = concatenate([inp1, inp2], axis=3) # If you use multiple inputs, you can concatenate them like this
    x = Conv2D(num_filt, 3, activation='relu', padding='same')(inputs)
    x = Conv2D(num_filt, 3, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filt * 2, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 2, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)
    x_2 = x
    
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)
    x_3 = x
    
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)
    x_4 = x
    
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(num_filt * 16, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 16, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)
    
    x = Conv2D(num_filt * 4, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x))
    x = concatenate([x_4,x], axis=3)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)

    x = Conv2D(num_filt * 4, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x))
    x = concatenate([x_3,x], axis=3)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 4, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)

    x = Conv2D(num_filt * 2, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x))
    x = concatenate([x_2,x], axis=3)
    x = Conv2D(num_filt * 2, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt * 2, 3, activation='relu', padding='same')(x)
    x = Dropout(dropout_rate)(x)

    x = Conv2D(num_filt, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(x))
    x = concatenate([inputs, x], axis=3)
    x = Conv2D(num_filt, 3, activation='relu', padding='same')(x)
    x = Conv2D(num_filt, 3, activation='relu', padding='same')(x)
    x = Conv2D(1, 1, activation="sigmoid")(x)

    model = Model(inputs=[inputs], outputs=x)
    return model

# NOTE: The generator is a U-Net originally proposed for segmentation tasks.
# NOTE: The model has 4 down- and upsampling layers. Maybe similar to what you have used in Assignment 2. Is it "deep" enough?
# NOTE: Which layers are important for the model? Which layers are important for uncertainty estimation?

## The below cell is all the code you are asked to modify. Change the input contrasts, learning rates and the optimizers to achieve the requirements.

In [ ]:
class MonteCarloDropoutModel(object):
    def __init__(self,model):
        self.f = Model(model.inputs, model.layers[-1].output)

    def predict(self,x, n_iter=5):
        result = []
        for _ in range(n_iter):
            result.append(self.f([x], training=True))
        result = tf.math.reduce_std(tf.stack(result, axis=0), axis=0)
        return result

# NOTE: No need to change this model. It's an implementation of a Monte Carlo Dropout Uncertainty estimation model.
# NOTE: This model will predict using your trained model 'n_iter' times, and return the standard deviation of the outputs.
# NOTE: The 'training=True' part ensures that the dropout layers are used in all 'n_iter' predictions.
# NOTE: This means the outputs will be slightly different.

In [ ]:
def dice_coef(y_true, y_pred, smooth=0.1):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# NOTE: An implementation of the DICE score, and the corresponding Dice loss, ready for training.

In [ ]:
generator = build_generator()
generator.summary()
generator_MCunc = MonteCarloDropoutModel(generator)

learning_rate = 0.01
generator.compile(loss=dice_coef_loss, metrics=dice_coef, optimizer=Adam(lr=learning_rate))
# NOTE: After building the model, you also need to build the uncertainty model. No need to change anything about that part.
# NOTE: Motivate your selection of the optimizer and learning rate.

In [ ]:
n_epochs = 3

plot_validating_loss = []
plot_validating_unc = []
for epoch in range(n_epochs):
    training_loss = []
    validating_loss = []
    validating_unc = []
    
    for idx, (t2, mask) in enumerate(gen_train): # Add the additional inputs inside the brackets
        h = generator.train_on_batch([t2], mask)
        training_loss.append(h)
    
    for idx, (t2, mask) in enumerate(gen_val): # Add the additional inputs inside the brackets
        validating_loss.append(generator.test_on_batch([t2], mask)[1])
        validating_unc.append(np.mean(generator_MCunc.predict([t2]).numpy()))
    
    plot_validating_loss.append(np.mean(validating_loss))
    plot_validating_unc.append(np.mean(validating_unc))
        
    print(np.mean(validating_loss))
    print(np.mean(validating_unc))
        

    
    clear_output(wait=True)
    print("Epoch " + str(epoch) + ", validation loss: " + str(np.mean(validating_loss))[:6])
    fig, axs = plt.subplots(1, 2, figsize=(16,8))
    axs[0].plot(np.linspace(0, epoch, epoch + 1), plot_validating_loss, label="val_loss")
    axs[1].plot(np.linspace(0, epoch, epoch + 1), plot_validating_unc, label="val_unc")
    axs[0].set_title("Validation loss")
    axs[0].grid(True)
    axs[0].set_xlabel('Epoch')
    axs[1].set_title("MC Uncertainty")
    axs[1].grid(True)
    axs[1].set_xlabel('Epoch')
    plt.show()
    
    # if (np.mean(validating_loss) < 0.015):
    #     break

# NOTE: When should training stop? How long did it take to reach the required DICE score?
# NOTE: Describe what behavior you expect from the two plots?
# NOTE: Detail what outcomes you have faced when the training failed? Why do you think that happened? How did you try to fix it?

In [ ]:
inp, mask = gen_val[np.random.randint(0, len(gen_val))] # Add the additional inputs
prediction = generator.predict([inp])
uncertainty = generator_MCunc.predict([inp])
plt.figure(figsize=(8, 4 * batch_size))
for idx in range(batch_size):
    plt.subplot(batch_size, 4, idx * 4 + 1)
    plt.imshow(inp[idx, :, :, 1], cmap='gray', vmin=0, vmax=1)
    plt.colorbar()
    plt.title('Image', fontsize=22)
    plt.subplot(batch_size, 4, idx * 4 + 2)
    plt.imshow(mask[idx, :, :], cmap='gray', vmin=0, vmax=1)
    plt.colorbar()
    plt.title('GT', fontsize=22)
    plt.subplot(batch_size, 4, idx * 4 + 3)
    plt.imshow(prediction[idx, :, :], cmap='gray', vmin=0, vmax=1)
    plt.colorbar()
    plt.title('PRED', fontsize=22)
    plt.subplot(batch_size, 4, idx * 4 + 4)
    plt.imshow(uncertainty[idx, :, :], cmap='Reds', vmin=0, vmax=np.max(uncertainty))
    plt.colorbar()
    plt.title('MC-uncertainty', fontsize=22)

# NOTE: How good do the images look like? What do you think is needed for better results?
# NOTE: Discuss the difference between false positives and false negatives in the segmentations.
# NOTE: Discuss the uncertainties. Where is it large, where is it small? Why? Is this what you would expect?

In [ ]:
testing_loss = []
for idx, (t2, mask) in enumerate(gen_test): # Add the additional inputs inside the brackets
    mn = np.max(generator_MCunc.predict([t2]), axis=(1, 2, 3))
    testing_loss.extend(mn)
plt.hist(testing_loss)

# NOTE: Only evaluate the testing set, when you are not changing the code anymore.
# NOTE: How different is the performance on the validation and testing sets?
# NOTE: Do your results speak of overfitting? Underfitting?
# NOTE: This code evaluates the uncertainty maps. 
# NOTE: Explore the histogram, and investigate why some data samples might have larger uncertainties